# AetherMind JEPA Simulation
This notebook demonstrates the **Joint Embedding Predictive Architecture (JEPA)** controlling a simulated vehicle.
The agent learns to predict the physics of the world in real-time. High prediction error ("Energy") represents surprise, triggering a safety reflex (Braking).

In [ ]:
import sys
import os
import numpy as np
import time

# Add project root to sys.path
sys.path.append(os.path.abspath(os.getcwd()))

try:
    from brain.jepa_aligner import JEPAAligner
    print("Success: AetherMind modules imported.")
except ImportError:
    print("Error: Could not import AetherMind modules. Ensure you are running this from the project root.")

## Define Simulation World
A simple class to simulate a vehicle moving towards a wall. It includes a "Sudden Surprise" event at step 30 to test the JEPA's reaction.

In [ ]:
class SimpleVehicleSimulator:
    def __init__(self):
        self.distance_to_wall = 100.0
        self.speed = 2.0
        self.step = 0
    
    def update(self, action):
        """
        Updates the vehicle state based on action.
        Action: 'accel' (+1 speed), 'brake' (-1 speed), 'maintain' (0 change)
        """
        self.step += 1
        
        # Physics
        if action == 'accel':
            self.speed += 0.5
        elif action == 'brake':
            self.speed -= 1.0
            
        self.speed = max(0.0, self.speed) # No reverse
        
        # Move
        self.distance_to_wall -= self.speed
        
        # Surprise Event at step 30 (Erratic slip or sudden obstacle jump)
        if self.step == 30:
            self.distance_to_wall -= 20 # Sudden jump closer!
            
        return self.get_state()
    
    def get_state(self):
        # Return normalized vector [distance_norm, speed_norm]
        # Dimensions matching JEPA (let's use 16 as define in app.py or generic 1024)
        # We'll use small dimension for demo
        vec = np.zeros(16)
        vec[0] = max(0, self.distance_to_wall) / 100.0
        vec[1] = self.speed / 10.0
        return vec

## Initialize Agent & Environment

In [ ]:
# Initialize JEPA Brain
# Using 16 dimensions as per our text simulation standards
jepa = JEPAAligner(dimension=16, energy_threshold=0.1)

# Initialize Simulator
sim = SimpleVehicleSimulator()
last_state = np.zeros(16)

print("System Initialized.")

## Run Simulation Loop
The loop runs for 50 ticks.
Observe how the **Energy** (Surprise) spikes at Step 30, causing the Agent to switch from `maintain` to `brake`.

In [ ]:
print(f"{'STEP':<6} {'DIST':<10} {'SPEED':<10} {'ENERGY':<10} {'ACTION'}")
print("-" * 50)

for i in range(51):
    # 1. Prediction (JEPA Energy)
    # How surprising is the current state given the last state?
    current_state_vec = sim.get_state()
    
    # Calculate Energy (Prediction Error)
    energy = jepa.compute_energy(last_state, current_state_vec)
    
    # 2. Learning
    # Update internal world model to predict this transition next time
    jepa.update_world_model(last_state, current_state_vec, learning_rate=0.1)
    
    # 3. Decision Logic based on "Surprise" (Energy)
    action = "maintain"
    
    if energy > 0.4:
        action = "brake" # Surprise! Slow down to gather info/safety
    elif sim.distance_to_wall < 10:
        action = "brake" # Hard safety limit
    elif sim.distance_to_wall > 20:
        action = "accel"
        
    # 4. Execute Action
    last_state = current_state_vec
    sim.update(action)
    
    # Log
    dist_val = current_state_vec[0] * 100
    speed_val = current_state_vec[1] * 10
    print(f"{i:<6} {dist_val:<10.1f} {speed_val:<10.1f} {energy:<10.4f} {action}")
    
    if dist_val <= 0:
        print("\n💥 CRASH! Simulation Ended.")
        break